<a href="https://colab.research.google.com/github/s-grzhang/gwc-chatbot/blob/main/gwc_transfer_learning_k_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
import numpy as np

# Load GPT-2 small model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Add a padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Ensure that the pad_token is set
tokenizer.pad_token = tokenizer.eos_token

print(f"Padding token added: {tokenizer.pad_token}")

# Load the dataset
file_path = '/content/drive/My Drive/gwc_chatbot_data_revised.csv'
df = pd.read_csv(file_path)

# Convert to Hugging Face Dataset
def df_to_dataset(df):
    return Dataset.from_pandas(df)

dataset = df_to_dataset(df)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['User Input'], truncation=True, padding='max_length')

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Define a Custom Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        item = {}
        for key in self.dataset.features:
            value = self.dataset[key][idx]
            if isinstance(value, str):
                # Tokenize the string value and convert to tensor
                tokenized_value = tokenizer(value, truncation=True, padding='max_length', return_tensors='pt')
                item['input_ids'] = tokenized_value['input_ids'].squeeze(0)
                item['attention_mask'] = tokenized_value['attention_mask'].squeeze(0)
            elif isinstance(value, list):
                # Convert list of numeric values to tensor
                item[key] = torch.tensor(value)
            elif isinstance(value, (int, float)):
                # Convert single numeric value to tensor
                item[key] = torch.tensor([value])
            else:
                print(f"Unexpected data type for key '{key}' at index {idx}: {type(value)}")

        # Use input_ids as labels
        item['labels'] = item['input_ids'].clone()

        return item

    def __len__(self):
        return len(self.dataset)

# Define the data collator function
def data_collator(data):
    input_ids = torch.stack([f['input_ids'] for f in data])
    attention_mask = torch.stack([f['attention_mask'] for f in data])
    labels = input_ids.clone()  # Use input_ids as labels

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

# Set the number of folds for cross-validation
k = 5  # Define k for K-Fold cross-validation

# k-Fold Cross-Validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)
results = []
for fold, (train_index, val_index) in enumerate(kf.split(tokenized_dataset)):
    print(f"Training Fold {fold + 1}/{k}")

    # Split data into training and validation folds
    train_dataset = tokenized_dataset.select(train_index)
    val_dataset = tokenized_dataset.select(val_index)

    train_dataset = CustomDataset(train_dataset)
    val_dataset = CustomDataset(val_dataset)

    # Define the training arguments
    training_args = TrainingArguments(
        output_dir=f"./results_fold_{fold}",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=7,
        weight_decay=0.01,
        logging_dir=f'./logs_fold_{fold}',
        logging_steps=10,
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="loss",
    )

    # Define the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    results.append(eval_results)
    print(f"Fold {fold + 1} Evaluation Results: {eval_results}")

# Average results across all folds
average_results = {key: np.mean([result[key] for result in results]) for key in results[0]}
print(f"Average Results Across All Folds: {average_results}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

Padding token added: <|endoftext|>


Map:   0%|          | 0/98 [00:00<?, ? examples/s]

Training Fold 1/5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.231200,0.054481
2,0.039200,0.033145
3,0.026500,0.027754
4,0.022300,0.026049
5,0.017600,0.025164
6,0.015100,0.024565


In [ ]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/My Drive/gwc_transfer_learning_chatbot_model_k-fold')
tokenizer.save_pretrained('/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer_k-fold')

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel

# Load the tokenizer and model
tokenizer_name = '/content/drive/My Drive/gwc_transfer_learning_chatbot_tokenizer_k-fold'
model_name = '/content/drive/My Drive/gwc_transfer_learning_chatbot_model_k-fold'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Test the chatbot
def get_response(user_input):
    inputs = tokenizer.encode(user_input, return_tensors="pt")
    outputs = model.generate(inputs, max_length=63, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

user_input = "When does the first meeting take place?"
response = get_response(user_input)
print(response)
